IMPORTAR BIBLIOTECAS


In [1]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

LENDO O DATASET

In [2]:
# Caminho do dataset
path = "./dataset/heart_statlog_cleveland_hungary_final.csv"

# Lendo o CSV com o pandas
df = pd.read_csv(path)

EXPLORAÇÃO INICIAL

In [3]:
df.head()

,age,sex,chest pain type,resting bp s,cholesterol,fasting blood sugar,resting ecg,max heart rate,exercise angina,oldpeak,ST slope,target
0,40,1,2,140,289,0,0,172,0,0.0,1,0
1,49,0,3,160,180,0,0,156,0,1.0,2,1
2,37,1,2,130,283,0,1,98,0,0.0,1,0
3,48,0,4,138,214,0,0,108,1,1.5,2,1
4,54,1,3,150,195,0,0,122,0,0.0,1,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1190 entries, 0 to 1189
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  1190 non-null   int64  
 1   sex                  1190 non-null   int64  
 2   chest pain type      1190 non-null   int64  
 3   resting bp s         1190 non-null   int64  
 4   cholesterol          1190 non-null   int64  
 5   fasting blood sugar  1190 non-null   int64  
 6   resting ecg          1190 non-null   int64  
 7   max heart rate       1190 non-null   int64  
 8   exercise angina      1190 non-null   int64  
 9   oldpeak              1190 non-null   float64
 10  ST slope             1190 non-null   int64  
 11  target               1190 non-null   int64  
dtypes: float64(1), int64(11)
memory usage: 111.7 KB


In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,1190.0,53.720168,9.358203,28.0,47.0,54.0,60.00,77.0
sex,1190.0,0.763866,0.424884,0.0,1.0,1.0,1.00,1.0
chest pain type,1190.0,3.232773,0.935480,1.0,3.0,4.0,4.00,4.0
resting bp s,1190.0,132.153782,18.368823,0.0,120.0,130.0,140.00,200.0
cholesterol,1190.0,210.363866,101.420489,0.0,188.0,229.0,269.75,603.0
fasting blood sugar,1190.0,0.213445,0.409912,0.0,0.0,0.0,0.00,1.0
resting ecg,1190.0,0.698319,0.870359,0.0,0.0,0.0,2.00,2.0
max heart rate,1190.0,139.732773,25.517636,60.0,121.0,140.5,160.00,202.0
exercise angina,1190.0,0.387395,0.487360,0.0,0.0,0.0,1.00,1.0
oldpeak,1190.0,0.922773,1.086337,-2.6,0.0,0.6,1.60,6.2


In [6]:
df.shape

(1190, 12)

In [7]:
df['target'].value_counts(normalize=True)

target
1    0.528571
0    0.471429
Name: proportion, dtype: float64

LIMPANDO OS DADOS

In [8]:
# Verificando a porcentagem de linhas contendo valores igual a 0 da coluna 'cholesterol'
round(sum(df['cholesterol'] == 0) / len(df) * 100, 1)

14.5

In [9]:
# Verificando a média da coluna cholesterol
df_sem_zero_chol = df[df.cholesterol != 0]
df_sem_zero_chol['cholesterol'].describe()

count    1018.000000
mean      245.906680
std        57.244599
min        85.000000
25%       209.000000
50%       240.000000
75%       276.000000
max       603.000000
Name: cholesterol, dtype: float64

In [10]:
# Substituindo as linhas com valor 0 pelo valor médio da coluna cholesterol
df['cholesterol'] = df['cholesterol'].replace(0, 240)

In [11]:
# Verificando a porcentagem de linhas contendo valores igual a 0 da coluna 'resting bp s'
round(sum(df['resting bp s'] == 0) / len(df) * 100, 3)

0.084

In [12]:
# Identificando as linhas com valores 0 da coluna 'resting bp s'
df[df['resting bp s'] == 0]

,age,sex,chest pain type,resting bp s,cholesterol,fasting blood sugar,resting ecg,max heart rate,exercise angina,oldpeak,ST slope,target
450,55,1,3,0,240,0,0,155,0,1.5,2,1


In [13]:
# Verificando a média da coluna 'resting bp s'
df['resting bp s'].describe()

count    1190.000000
mean      132.153782
std        18.368823
min         0.000000
25%       120.000000
50%       130.000000
75%       140.000000
max       200.000000
Name: resting bp s, dtype: float64

In [14]:
# Substituindo as linhas com valor 0 pelo valor médio da coluna resting bp s
df['resting bp s'] = df['resting bp s'].replace(0, 130)

In [15]:
# Verificando a porcentagem de linhas com valores abaixo de 0 da coluna 'oldpeak'
round(sum(df['oldpeak'] < 0) / len(df) * 100, 1)

1.1

In [16]:
# Verificando a média da coluna 'oldpeak'
df['oldpeak'].describe()

count    1190.000000
mean        0.922773
std         1.086337
min        -2.600000
25%         0.000000
50%         0.600000
75%         1.600000
max         6.200000
Name: oldpeak, dtype: float64

In [17]:
# Substituindo os valores abaixo de 0 pela média da coluna 'oldpeak'
df[df < 0] = 0.6

In [18]:
# Extrai os dados de entrada (X), rótulos (y) e outras informações
X = df.drop(columns=['target'])
y = df['target']
y_names = 'target'

# Cria um DataFrame do pandas com os dados e os rótulos
df = pd.DataFrame(X)
df[y_names] = y

df = df.dropna()

# Retorna o Dataframe contendo os dados 
df

,age,sex,chest pain type,resting bp s,cholesterol,fasting blood sugar,resting ecg,max heart rate,exercise angina,oldpeak,ST slope,target
0,40,1,2,140,289,0,0,172,0,0.0,1,0
1,49,0,3,160,180,0,0,156,0,1.0,2,1
2,37,1,2,130,283,0,1,98,0,0.0,1,0
3,48,0,4,138,214,0,0,108,1,1.5,2,1
4,54,1,3,150,195,0,0,122,0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1185,45,1,1,110,264,0,0,132,0,1.2,2,1
1186,68,1,4,144,193,1,0,141,0,3.4,2,1
1187,57,1,4,130,131,0,0,115,1,1.2,2,1
1188,57,0,2,130,236,0,2,174,0,0.0,2,1


ANTES DA NORMALIZAÇÃO

In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Inicializa o modelo KNN
knn = KNeighborsClassifier()

# Treina o modelo com os dados de entrada X e os rótulos y
knn.fit (X, y)

# Realiza previsões com o modelo treinado
y_prevs = knn.predict (X)

# Calcula a acurácia das previsões
accuracy_score(y, y_prevs)

0.7890756302521008

PÓS NORMALIZAÇÃO I com train_test_split

In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# Dividir o conjunto de dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar e ajustar o scaler aos dados de treinamento e transformar os dados de treinamento e teste
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Definir a grade de hiperparâmetros para a busca
param_grid = {'n_neighbors': [3, 5, 7, 9, 11],
              'weights': ['uniform', 'distance'],
              'metric': ['euclidean', 'manhattan', 'chebyshev']
}

# Inicializar o objeto de busca de grade com validação cruzada
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)

# Realizar a busca de hiperparâmetros usando validação cruzada no conjunto de treinamento
grid_search.fit(X_train_scaled, y_train)

# Obter os melhores hiperparâmetros encontrados pela busca
best_params = grid_search.best_params_

# Criar o modelo KNN com os melhores hiperparâmetros e ajustar aos dados de treinamento
best_knn = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'])
best_knn.fit(X_train_scaled, y_train)

# Fazer previsões nos dados de teste com o modelo ajustado
y_preds = best_knn.predict(X_test_scaled)

# Calcular a acurácia do modelo usando os dados de teste
accuracy_score(y_test, y_preds)

0.8697478991596639

PÓS NORMALIZAÇÃO II com MinMaxScaler

In [21]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV, cross_val_score

# Inicializar o StandardScaler para padronização dos dados
scaler = StandardScaler()

# Padronizar as características (atributos)
X_scaled = scaler.fit_transform(X)

# Inicializar o MinMaxScaler para normalização dos dados
min_max_scaler = MinMaxScaler()

# Normalizar as características (atributos)
X_normalized = min_max_scaler.fit_transform(X)

# Definir a grade de hiperparâmetros para a busca
param_grid = {'n_neighbors': [3, 5, 7, 9, 11], # Número de vizinhos
              'weights': ['uniform', 'distance'], # Método de ponderação dos vizinhos
              'metric': ['euclidean', 'manhattan', 'chebyshev'] # Métrica de distância
}

# Inicializar o objeto de busca de grade com validação cruzada
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)

# Realizar a busca de hiperparâmetros usando validação cruzada nos dados padronizados
grid_search.fit(X_scaled, y)

# Obter os melhores hiperparâmetros encontrados pela busca
best_params = grid_search.best_params_

# Criar o modelo KNN com os melhores hiperparâmetros
best_knn = KNeighborsClassifier(**best_params)

# Avaliar o modelo KNN utilizando validação cruzada nos dados padronizados
scores = cross_val_score(best_knn, X_scaled, y, cv=10)

# Calcular a média das pontuações de validação cruzada para obter a acurácia média do modelo
scores.mean()

0.9268907563025209

PÓS NORMALIZAÇÃO III com PCA

In [22]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

# Dividir os dados em características (X) e alvo (y)
X = df.drop(columns=['target'])
y = df['target']

# Definir os hiperparâmetros a serem ajustados
param_grid = {
    'pca__n_components': [0.95, 0.99],  # Reduzir para 95% ou 99% da variância explicada
    'knn__n_neighbors': [3, 5, 7, 9, 11],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan', 'chebyshev']
}

# Criar pipeline com PCA e KNN
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA()),
    ('knn', KNeighborsClassifier())
])

# Realizar a busca de hiperparâmetros utilizando validação cruzada
grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X, y)

# Melhores parâmetros encontrados
best_params = grid_search.best_params_

# Criar o modelo com os melhores parâmetros
best_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=best_params['pca__n_components'])),
    ('knn', KNeighborsClassifier(n_neighbors=best_params['knn__n_neighbors'],
                                  weights=best_params['knn__weights'],
                                  metric=best_params['knn__metric']))
])

# Avaliar o desempenho do modelo utilizando validação cruzada
scores = cross_val_score(best_pipeline, X, y, cv=10)

scores.mean()

0.9226890756302522